In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import transforms
from torch.autograd import Variable

In [11]:
class SE_Block(nn.Module):
    def __init__(self, ch_in, reduction=16):
        super(SE_Block, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # 全局自适应池化
        self.fc = nn.Sequential(
            nn.Linear(ch_in, ch_in // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(ch_in // reduction, ch_in, bias=False),
            nn.Sigmoid()
        )
 
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c) # squeeze操作
        y = self.fc(y).view(b, c, 1, 1) # FC获取通道注意力权重，是具有全局信息的
        return x * y.expand_as(x) # 注意力作用每一个通道上

In [13]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DepthwiseSeparableConv, self).__init__()
 
        self.depthwise_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels)
        self.pointwise_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)
        self.SE = SE_Block(out_channels)  
 
    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        x = self.relu(x)
        x = self.SE(x)
        return x

In [14]:
class MobileNetV1(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV1, self).__init__()
 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU(inplace=True)
 
        self.dw_separable_conv1 = DepthwiseSeparableConv(32, 64)
        self.dw_separable_conv2 = DepthwiseSeparableConv(64, 128)
        self.dw_separable_conv3 = DepthwiseSeparableConv(128, 128)
        self.dw_separable_conv4 = DepthwiseSeparableConv(128, 256)
        self.dw_separable_conv5 = DepthwiseSeparableConv(256, 256)
        self.dw_separable_conv6 = DepthwiseSeparableConv(256, 512)
        self.dw_separable_conv7 = DepthwiseSeparableConv(512, 512)
        self.dw_separable_conv8 = DepthwiseSeparableConv(512, 512)
        self.dw_separable_conv9 = DepthwiseSeparableConv(512, 512)
        self.dw_separable_conv10 = DepthwiseSeparableConv(512, 512)
        self.dw_separable_conv11 = DepthwiseSeparableConv(512, 512)
        self.dw_separable_conv12 = DepthwiseSeparableConv(512, 1024)
        self.dw_separable_conv13 = DepthwiseSeparableConv(1024, 1024)
 
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dw_separable_conv1(x)
        x = self.dw_separable_conv2(x)
        x = self.dw_separable_conv3(x)
        x = self.dw_separable_conv4(x)
        x = self.dw_separable_conv5(x)
        x = self.dw_separable_conv6(x)
        x = self.dw_separable_conv7(x)
        x = self.dw_separable_conv8(x)
        x = self.dw_separable_conv9(x)
        x = self.dw_separable_conv10(x)
        x = self.dw_separable_conv11(x)
        x = self.dw_separable_conv12(x)
        x = self.dw_separable_conv13(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
 
        return x

In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

In [16]:
train_dataset = datasets.ImageFolder(root='split-garbage-dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='split-garbage-dataset/val', transform=transform)

In [17]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [19]:
if __name__ == '__main__':
    # 如果可用的话，在设备上选择CUDA，否则使用CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 创建MobileNetV1模型并将其移动到所选设备上
    net = MobileNetV1(num_classes=6).to(device)
    # 定义损失函数并将其移动到设备上
    cross = nn.CrossEntropyLoss().to(device)
    
    # 定义优化器并将网络参数传递给它
    optimizer = torch.optim.SGD(net.parameters(), lr=0.05)  # 修复了优化器行
    
    for epoch in range(10):
        print(f"Epoch:{epoch}")
        for data in train_loader:
            inputs, labels = data
            # 将输入数据和标签移动到GPU
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net.forward(inputs)
            loss = cross(outputs, labels)
            loss.backward()
            optimizer.step()
            print(f"Batch完成，损失：{loss.item()}")
        print(f"Epoch 完成")


Epoch:0
Batch完成，损失：1.7966965436935425
Batch完成，损失：1.788130521774292
Batch完成，损失：1.795118808746338
Batch完成，损失：1.7900460958480835
Batch完成，损失：1.7893766164779663
Batch完成，损失：1.779841661453247
Batch完成，损失：1.7749309539794922
Batch完成，损失：1.7707849740982056
Batch完成，损失：1.775256633758545
Batch完成，损失：1.7834715843200684
Batch完成，损失：1.7587759494781494
Batch完成，损失：1.7723621129989624
Batch完成，损失：1.758064866065979
Batch完成，损失：1.7570552825927734
Batch完成，损失：1.7552388906478882
Batch完成，损失：1.7774779796600342
Batch完成，损失：1.7388148307800293
Batch完成，损失：1.7532241344451904
Batch完成，损失：1.7519505023956299
Batch完成，损失：1.7483924627304077
Batch完成，损失：1.7500866651535034
Batch完成，损失：1.7472838163375854
Batch完成，损失：1.7695616483688354
Batch完成，损失：1.7807888984680176
Batch完成，损失：1.7340295314788818
Batch完成，损失：1.7453923225402832
Batch完成，损失：1.779615044593811
Batch完成，损失：1.7331933975219727
Batch完成，损失：1.7578126192092896
Batch完成，损失：1.7738025188446045
Batch完成，损失：1.7545629739761353
Batch完成，损失：1.7381376028060913
Batch完成，损失：1.7148263454437256
Batch完成，

KeyboardInterrupt: 